In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from torchvision.io import read_image

import torch, torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

import copy
import time
import numpy as np

cudnn.benchmark = True

In [ ]:
use_cuda = True
device = 'cuda:0' if torch.cuda.is_available() and use_cuda else 'cpu'
print(device)

cuda:0


In [ ]:
!nvidia-smi

Mon Dec 12 12:39:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file, index_col=0).reset_index()
        self.img_filepath = self.img_labels.filepath
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_filepath.iloc[idx])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 2]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
transform = transforms.Compose(
    [transforms.Resize((224,224)), 
    # transforms.ToTensor(), # read_image로 가져와서 텐서임
    ])

batch_size = 64

In [ ]:
trainset = CustomImageDataset('/content/drive/MyDrive/csv_house/train.csv', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

valset = CustomImageDataset('/content/drive/MyDrive/csv_house/val.csv', transform=transform)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = CustomImageDataset('/content/drive/MyDrive/csv_house/test.csv', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, num_workers=2)

datasets = {'train': trainset, 'val': valset, 'test': testset}
dataloaders = {'train': trainloader, 'val': valloader, 'test': testloader}
datasets_sizes = {x: len(datasets[x]) for x in ['train', 'val', 'test']}


In [ ]:
pd.read_csv('/content/drive/MyDrive/csv_house/val.csv', index_col=0).reset_index().iloc[0, 2]

0

In [ ]:
valset = CustomImageDataset('/content/drive/MyDrive/csv_house/val.csv', transform=transform)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=2)

In [ ]:
val = pd.read_csv('/content/drive/MyDrive/csv_house/total.csv', index_col=0).reset_index()

In [ ]:
for ext in [val_path[-3:] for val_path in val.filepath]:
    if ext != 'png' and ext != 'jpg':
        print(ext)
    else:
        pass

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-'*10)

    # 각 epoch은 학습 단계와 검증 단계 가짐
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() # 모델 학습모드로 설정
            else:
                model.eval() # 모델 평가모드로 설정
      
            running_loss = 0.0
            running_corrects = 0

      # 데이터 반복
            batch_loss = 0.0
            for i, (inputs, labels) in enumerate(dataloaders[phase]):
                inputs = inputs.float().to(device)
                labels = labels.float().unsqueeze(1).to(device)

        # parameter gradient 0으로 설정
                optimizer.zero_grad()

        # 순전파
        # 학습 때만 연산 기록 추적
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

          # 학습 단계인 경우 역전파 + 최적화
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

          # 통계 ~ 여기에 학습 그래프 넣어보기
                # 배치 10개 당
                batch_loss += loss.item()
                if i % 10 == 9:
                    print(f'[{epoch+1}, {i+1:5d}] loss: {batch_loss / 10:.3f}')
                    batch_loss = 0.0
                # epoch 당
                running_loss += loss.item() * inputs.size(0)
                pred = torch.special.expit(outputs) >= torch.FloatTensor([0.5]).to(device)
                running_corrects += torch.sum(pred == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / datasets_sizes[phase]
            epoch_acc = running_corrects.double() / datasets_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # 모델 deepcopy
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

  # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
backbone = torchvision.models.efficientnet_v2_s(weights='EfficientNet_V2_S_Weights.IMAGENET1K_V1')

In [ ]:
for param in backbone.parameters():
    param.requires_grad = False

backbone.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(1280, 512),
    torch.nn.Linear(512, 128),
    torch.nn.Linear(128, 1),
)

model = backbone.to(device)
criterion = torch.nn.BCEWithLogitsLoss()

optimizer = torch.optim.SGD(model.classifier.parameters(), lr=1e-3, momentum=0.9)

# 7 에폭마다 0.1씩 학습률 감소
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=512, bias=True)
  (2): Linear(in_features=512, out_features=128, bias=True)
  (3): Linear(in_features=128, out_features=1, bias=True)
)

In [ ]:
for param in backbone.parameters():
    print(param.requires_grad)

In [40]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=2)

Epoch 1/2
----------
[1,    10] loss: 0.699
[1,    20] loss: 0.686
[1,    30] loss: 0.666
[1,    40] loss: 0.654
[1,    50] loss: 0.640
[1,    60] loss: 0.621
[1,    70] loss: 0.613
[1,    80] loss: 0.610
[1,    90] loss: 0.594
[1,   100] loss: 0.581
[1,   110] loss: 0.581
[1,   120] loss: 0.580
[1,   130] loss: 0.552
[1,   140] loss: 0.564
[1,   150] loss: 0.562
[1,   160] loss: 0.539
[1,   170] loss: 0.520
[1,   180] loss: 0.534
[1,   190] loss: 0.513
[1,   200] loss: 0.521
[1,   210] loss: 0.500
[1,   220] loss: 0.507
[1,   230] loss: 0.467
[1,   240] loss: 0.471
[1,   250] loss: 0.446
[1,   260] loss: 0.460
[1,   270] loss: 0.462
[1,   280] loss: 0.457
[1,   290] loss: 0.452
[1,   300] loss: 0.439
[1,   310] loss: 0.442
[1,   320] loss: 0.450
[1,   330] loss: 0.415
[1,   340] loss: 0.411
[1,   350] loss: 0.412
[1,   360] loss: 0.414
[1,   370] loss: 0.413
[1,   380] loss: 0.410
[1,   390] loss: 0.438
[1,   400] loss: 0.421
[1,   410] loss: 0.417
[1,   420] loss: 0.373
[1,   430] lo

In [ ]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f7e7bc00fd0>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7f7e7bc00d90>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f7e7bc00f10>}

In [42]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Project3/sh/1213_epochs2_weights.pth')